In [1]:
import os
os.chdir('/home/xyz/data/haoyan/bandits')

import bandits as bd
import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

/home/xyz/miniconda3/envs/bandit/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_csv('/home/xyz/data/haoyan/bandits/data/Walmart_sales.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data['month'] = data['Date'].dt.month

x = data[['Store', 'month', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']]
y = data['Weekly_Sales']

x_train, x_test, y_train, y_test=train_test_split(x,y,train_size=0.67,random_state=34)

In [3]:
def g(
    max_bin,
    n_estimators,
    max_depth,
    colsample_bytree,
    learning_rate,
    reg_alpha,
    reg_lambda,
):
    config = {
        "max_bin": max_bin,
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "colsample_bytree": min(colsample_bytree, 1),
        "learning_rate": learning_rate,
        "reg_alpha": reg_alpha,
        "reg_lambda": reg_lambda,
        "random_state": 34,
        "tree_method": "hist",
        "device": 'gpu',
    }

    model = XGBRegressor(**config)
    model.fit(x_train, y_train, verbose=1)
    predictions = model.predict(x_test)
    return -mean_absolute_error(y_test, predictions) / 1e5

In [4]:
n_experiment = 10

# gradient bandit hybrid

In [ ]:
params_list_dict = {
    "max_bin": [2**6, 2**7, 2**8, 2**9],
    "max_depth": [4, 5, 6, 7, 8, 9],
    "n_estimators": [500, 1000, 2500, 5000],
}

pbounds={
    "colsample_bytree": [0.2, 1],
    'learning_rate': [0.001, 0.5],
    'reg_alpha': [0, 3],
    'reg_lambda': [0, 3],
}

scores_list = []
actions_list = []

def single_func(n, f):
    bandit = bd.GaussianBandit(
        g,
        params_list_dict=params_list_dict,
        pbounds=pbounds,
        random_state=n*128,
        init_points=2,
        n_iter=2,
    )

    policy = bd.SoftmaxPolicy()
    agent = bd.GradientAgent(bandit, policy, alpha=1.5, avg_coef=None, baseline=True, increase_rate=0)
    env = bd.Environment(bandit, agent, 'Gradient Agents', random_state=n*128)
    scores, actions = env.run(10000, file=f)
    
    return scores, actions

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/rl_hybrid_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/rl_hybrid_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/rl_hybrid_results.txt', 'a') as f:
    res = [single_func(x, f) for x in range(n_experiment)]

0
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------


/home/xyz/miniconda3/envs/bandit/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:01:18] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


| 1         | -2.049    | 0.3127    | 0.01149   | 2.364     | 0.06154   |
| 2         | -0.6431   | 0.5548    | 0.2162    | 2.236     | 2.554     |
1
((256, 5, 500), {'colsample_bytree': 0.5547618294926189, 'learning_rate': 0.21615296658967176, 'reg_alpha': 2.2360298672621006, 'reg_lambda': 2.5537011636738054}) -0.6431458740112994
1
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.633    | 0.7642    | 0.2121    | 1.753     | 1.082     |
| 2         | -0.6738   | 0.9177    | 0.4488    | 2.995     | 0.8857    |
1.0
((256, 9, 500), {'colsample_bytree': 0.7642297233416242, 'learning_rate': 0.2120750598283257, 'reg_alpha': 1.7527879369688546, 'reg_lambda': 1.0823565186081896}) -0.6329581616172316
2
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.7066   | 0.7262 

| 1         | -0.6416   | 0.8232    | 0.4884    | 2.711     | 2.784     |
| 2         | -0.7375   | 0.4003    | 0.1004    | 2.079     | 1.822     |
1.0
((512, 8, 500), {'colsample_bytree': 0.8231669172866358, 'learning_rate': 0.48837353491192137, 'reg_alpha': 2.710711966799318, 'reg_lambda': 2.783997951262565}) -0.6415793747881355
14
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.6899   | 0.9424    | 0.4951    | 1.1       | 2.12      |
| 2         | -0.7696   | 0.568     | 0.4264    | 0.4441    | 0.7082    |
1.0
((64, 5, 2500), {'colsample_bytree': 0.942376712589795, 'learning_rate': 0.4951065086334376, 'reg_alpha': 1.1000568984696792, 'reg_lambda': 2.119566879710282}) -0.6898806268596985
15
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.7701   | 0.4203 

| 1         | -0.8572   | 0.2187    | 0.3366    | 0.6642    | 0.609     |
| 2         | -0.7682   | 0.686     | 0.4419    | 2.617     | 0.9095    |
1.0
((64, 7, 1000), {'colsample_bytree': 0.6860490367156582, 'learning_rate': 0.4419261897195094, 'reg_alpha': 2.616708944345058, 'reg_lambda': 0.9094649194973576}) -0.7681669935852166
27
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.6179   | 0.7821    | 0.08857   | 1.328     | 0.3323    |
| 2         | -0.6233   | 0.6927    | 0.1564    | 2.511     | 2.621     |
1.0
((512, 8, 2500), {'colsample_bytree': 0.7821060804931086, 'learning_rate': 0.08856898088342291, 'reg_alpha': 1.3280620654409223, 'reg_lambda': 0.3323409394227329}) -0.6178584455626177
28
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 3         | -0.6173   | 0.9

...

| 35        | -0.6927   | 0.5656    | 0.2137    | 1.228     | 2.969     |
| 36        | -0.6539   | 0.9547    | 0.01397   | 1.06      | 2.971     |
1.0
((64, 4, 2500), {'colsample_bytree': 0.7547776086423195, 'learning_rate': 0.03564519086221022, 'reg_alpha': 2.2030527768383337, 'reg_lambda': 2.305049103244502}) -0.6188503287429379
2481
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 65        | -0.6568   | 0.7515    | 0.3899    | 1.662     | 1.775     |
| 66        | -0.6726   | 0.5131    | 0.1933    | 2.871     | 2.841     |
1.0
((256, 7, 5000), {'colsample_bytree': 0.9683223053279717, 'learning_rate': 0.011976172760888124, 'reg_alpha': 1.7903534674364998, 'reg_lambda': 1.7987156644166669}) -0.5859384104048965
2482
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 53        | -0.6801  

| 55        | -0.6726   | 0.7555    | 0.4672    | 1.15      | 0.7178    |
| 56        | -0.589    | 1.0       | 0.0656    | 1.727     | 2.849     |
1.0
((256, 5, 1000), {'colsample_bytree': 1.0, 'learning_rate': 0.06560391068848395, 'reg_alpha': 1.7268167681940236, 'reg_lambda': 2.8490992838662255}) -0.5889876696916195
2494
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 49        | -0.8719   | 0.2274    | 0.4036    | 2.585     | 2.744     |
| 50        | -0.6301   | 0.8648    | 0.01604   | 1.506     | 1.26      |
1.0
((512, 4, 2500), {'colsample_bytree': 0.7292286167716509, 'learning_rate': 0.06985177404632681, 'reg_alpha': 0.45958283036039527, 'reg_lambda': 1.297496167747174}) -0.5946791904896422
2495
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 27        | -2.588    | 0.5782    |

| 55        | -0.7005   | 0.5949    | 0.5       | 2.138     | 3.0       |
| 56        | -0.7278   | 0.3068    | 0.2772    | 0.263     | 0.2349    |
1.0
((128, 4, 1000), {'colsample_bytree': 0.8570645026692294, 'learning_rate': 0.12109947774655104, 'reg_alpha': 1.3359950642178497, 'reg_lambda': 1.8370103609332546}) -0.6278237822740114
2507
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 43        | -0.6196   | 0.8715    | 0.1454    | 2.875     | 2.831     |
| 44        | -0.8199   | 0.6567    | 0.5       | 2.578     | 2.628     |
1.0
((128, 9, 500), {'colsample_bytree': 0.9372939420880793, 'learning_rate': 0.053911160048767184, 'reg_alpha': 2.510421958298305, 'reg_lambda': 2.4733302520050513}) -0.6098819457980226
2508
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 55        | -0.6693  

| 61        | -0.6354   | 0.9434    | 0.2452    | 1.661     | 1.331     |
| 62        | -0.6524   | 0.8259    | 0.3079    | 0.8438    | 2.399     |
1.0
((128, 8, 2500), {'colsample_bytree': 0.8368038170665046, 'learning_rate': 0.07374363104068064, 'reg_alpha': 0.751927515242911, 'reg_lambda': 2.6575094813483147}) -0.6089564663724106
2520
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 45        | -3.775    | 0.4772    | 0.001     | 2.864     | 2.498     |
| 46        | -0.8376   | 0.5466    | 0.4446    | 1.973     | 1.865     |
1.0
((128, 9, 500), {'colsample_bytree': 0.9372939420880793, 'learning_rate': 0.053911160048767184, 'reg_alpha': 2.510421958298305, 'reg_lambda': 2.4733302520050513}) -0.6098819457980226
2521
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 61        | -0.6311   

| 53        | -0.6934   | 0.351     | 0.0475    | 1.777     | 0.8864    |
| 54        | -0.8664   | 0.2511    | 0.3499    | 0.4479    | 2.727     |
1.0
((512, 4, 2500), {'colsample_bytree': 0.7292286167716509, 'learning_rate': 0.06985177404632681, 'reg_alpha': 0.45958283036039527, 'reg_lambda': 1.297496167747174}) -0.5946791904896422
2533
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 43        | -0.6504   | 1.0       | 0.5       | 0.9027    | 2.007     |
| 44        | -0.6204   | 0.8374    | 0.1933    | 0.5139    | 1.569     |
1.0
((256, 8, 1000), {'colsample_bytree': 0.7918710433705916, 'learning_rate': 0.06754995334027572, 'reg_alpha': 1.6164484258083962, 'reg_lambda': 2.355262233187915}) -0.5885189837452919
2534
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 63        | -0.5907  

In [6]:
scores_list = []
actions_list = []
for x in res:
    scores_list.append(x[0])
    actions_list.append(x[1])

NameError: name 'res' is not defined

In [ ]:
for scores, actions in zip(scores_list, actions_list):
    print(actions[np.argmax(scores)], scores[np.argmax(scores)], len(actions))

# gradient bandit random search

In [7]:
params_list_dict = {
    "max_bin": [2**4, 2**5, 2**6, 2**7, 2**8, 2**9, 2**10],
    "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "n_estimators": [100, 500, 1000, 2500, 5000],
}

pbounds={
    "colsample_bytree": [0.2, 1],
    'learning_rate': [0.001, 0.5],
    'reg_alpha': [0, 3],
    'reg_lambda': [0, 3],
}

scores_list = []
actions_list = []

def single_func(n, f):
    bandit = bd.GaussianBanditRandomSearch(
        g,
        params_list_dict=params_list_dict,
        pbounds=pbounds,
        random_state=n*128,
        max_single_discrete_param_n_iter=300,
    )

    policy = bd.SoftmaxPolicy()
    agent = bd.GradientAgent(bandit, policy, alpha=0.5, avg_coef=None, baseline=True, increase_rate=0)
    env = bd.Environment(bandit, agent, 'Gradient Agents', random_state=n*128)
    scores, actions = env.run(600, file=f)
    
    return scores, actions

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/rl_random_search_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/rl_random_search_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/rl_random_search_results.txt', 'a') as f:
    res = [single_func(x, f) for x in range(n_experiment)]

0
0.01
((256, 3, 5000), ({'colsample_bytree': 0.9088623519724577, 'learning_rate': 0.09252775479126046, 'reg_alpha': 2.661254733388616, 'reg_lambda': 0.16962686687613882}, {'max_bin': 256, 'max_depth': 3, 'n_estimators': 5000})) -0.6564767188301849
1
0.01
((16, 4, 5000), ({'colsample_bytree': 0.47552555662018686, 'learning_rate': 0.036370488973076986, 'reg_alpha': 1.1698756365655822, 'reg_lambda': 1.14154324892839}, {'max_bin': 16, 'max_depth': 4, 'n_estimators': 5000})) -1.1774656444275247
2
0.01
((64, 4, 100), ({'colsample_bytree': 0.7044285708342211, 'learning_rate': 0.0931157957153011, 'reg_alpha': 1.665212254259142, 'reg_lambda': 1.0992101779687005}, {'max_bin': 64, 'max_depth': 4, 'n_estimators': 100})) -1.115862739277307
3


KeyboardInterrupt: 

# random search

In [ ]:
import time
import numpy as np
import scipy.stats as stats

from sklearn.model_selection import ParameterSampler


# specify parameters and distributions to sample from
params_list_dict = {
    "max_bin": [2**6, 2**7, 2**8, 2**9],
    "max_depth": [4, 5, 6, 7, 8, 9],
    "n_estimators": [500, 1000, 2500, 5000],
    "colsample_bytree": stats.uniform(0.2, 1),
    'learning_rate': stats.uniform(0.001, 0.5),
    'reg_alpha': stats.uniform(0, 3),
    'reg_lambda': stats.uniform(0, 3),
}

# n_iter = max([len(actions) for actions in actions_list])
n_iter = 3000
n_experiment = 10

res_dict_list = []

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/grid_search_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/grid_search_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/grid_search_results.txt', 'a') as f:
    for n in range(n_experiment):
        param_list = list(ParameterSampler(params_list_dict, n_iter=n_iter, random_state=n*128))
        res_dict = {}
        for i, param in enumerate(param_list):
            start = time.time()
            res_dict[str(param)] = g(**param)
            end = time.time()
            f.write(f'({n*128}, {i}, {param}, {res_dict[str(param)]}, {end-start})\n')
            if i % 10 == 0:
                print(i, param, res_dict[str(param)])
            
        res_dict_list.append(res_dict)

0 {'colsample_bytree': 0.7488135039273247, 'learning_rate': 0.35859468318620974, 'max_bin': 512, 'max_depth': 7, 'n_estimators': 5000, 'reg_alpha': 2.5417552163523762, 'reg_lambda': 1.870691090357917} -0.6287757303672316
10 {'colsample_bytree': 0.8531083254653984, 'learning_rate': 0.12764580126989106, 'max_bin': 512, 'max_depth': 8, 'n_estimators': 5000, 'reg_alpha': 1.8705303033956047, 'reg_lambda': 1.0140228445166752} -0.6115529610493173
20 {'colsample_bytree': 0.8994792753175043, 'learning_rate': 0.14971847542756683, 'max_bin': 128, 'max_depth': 5, 'n_estimators': 1000, 'reg_alpha': 2.026317243600708, 'reg_lambda': 0.7493888269649343} -0.6337504031455979
30 {'colsample_bytree': 1.1441235194354547, 'learning_rate': 0.3759996243850502, 'max_bin': 256, 'max_depth': 6, 'n_estimators': 500, 'reg_alpha': 0.1630139650177609, 'reg_lambda': 0.5999895746892002} -0.6400580460716808
40 {'colsample_bytree': 0.8798794564067696, 'learning_rate': 0.229064886061431, 'max_bin': 512, 'max_depth': 8, '

370 {'colsample_bytree': 0.3709348059841045, 'learning_rate': 0.43383830473313767, 'max_bin': 64, 'max_depth': 6, 'n_estimators': 500, 'reg_alpha': 0.6917839099761645, 'reg_lambda': 1.0320305269525618} -0.8285873778127206
380 {'colsample_bytree': 0.6638946746593173, 'learning_rate': 0.3429898056832376, 'max_bin': 64, 'max_depth': 7, 'n_estimators': 1000, 'reg_alpha': 2.0222318669731316, 'reg_lambda': 0.9683200310410408} -0.7130397387461745
390 {'colsample_bytree': 0.9664978735303262, 'learning_rate': 0.2887020044510637, 'max_bin': 128, 'max_depth': 7, 'n_estimators': 1000, 'reg_alpha': 1.831689736331617, 'reg_lambda': 1.6769749086907795} -0.6501269293785311
400 {'colsample_bytree': 0.5843537322902954, 'learning_rate': 0.483046471410708, 'max_bin': 128, 'max_depth': 8, 'n_estimators': 500, 'reg_alpha': 1.6284194062584407, 'reg_lambda': 2.142161355221659} -0.7159806871998589
410 {'colsample_bytree': 1.1406386129939068, 'learning_rate': 0.05777018120427663, 'max_bin': 512, 'max_depth': 5,

740 {'colsample_bytree': 1.1625743710919205, 'learning_rate': 0.09655060030758356, 'max_bin': 256, 'max_depth': 9, 'n_estimators': 5000, 'reg_alpha': 2.5742811646675965, 'reg_lambda': 2.7719461496769857} -0.6217584666872646
750 {'colsample_bytree': 0.5572562506685832, 'learning_rate': 0.37439070986477285, 'max_bin': 64, 'max_depth': 6, 'n_estimators': 1000, 'reg_alpha': 0.8858525449287494, 'reg_lambda': 2.7234769348540775} -0.7239389308847545
760 {'colsample_bytree': 0.31239977810832803, 'learning_rate': 0.16701153124329104, 'max_bin': 256, 'max_depth': 9, 'n_estimators': 5000, 'reg_alpha': 1.2606694813605075, 'reg_lambda': 2.614143064784211} -0.7918946081700211
770 {'colsample_bytree': 0.8116296572395705, 'learning_rate': 0.2350882969250791, 'max_bin': 128, 'max_depth': 5, 'n_estimators': 2500, 'reg_alpha': 1.1001244188279826, 'reg_lambda': 0.127575262577726} -0.6567594649099577
780 {'colsample_bytree': 0.7034686093793188, 'learning_rate': 0.4612736913306111, 'max_bin': 256, 'max_dept

1110 {'colsample_bytree': 0.2731332895906477, 'learning_rate': 0.04691984981738895, 'max_bin': 128, 'max_depth': 5, 'n_estimators': 5000, 'reg_alpha': 1.5694611772612896, 'reg_lambda': 2.2432255488462407} -0.8819155737038018
1120 {'colsample_bytree': 1.0569970748307678, 'learning_rate': 0.05651300770395662, 'max_bin': 64, 'max_depth': 6, 'n_estimators': 1000, 'reg_alpha': 0.36193454681465675, 'reg_lambda': 0.10359315464002372} -0.6335064028866524
1130 {'colsample_bytree': 0.8205072571476393, 'learning_rate': 0.04371214648557181, 'max_bin': 256, 'max_depth': 8, 'n_estimators': 1000, 'reg_alpha': 2.289037884757505, 'reg_lambda': 1.351216448563162} -0.6068451323416902
1140 {'colsample_bytree': 0.6684823968620872, 'learning_rate': 0.019357684214107995, 'max_bin': 256, 'max_depth': 4, 'n_estimators': 5000, 'reg_alpha': 1.3823470889807514, 'reg_lambda': 1.4243004519285472} -0.622877937851636
1150 {'colsample_bytree': 0.3725776487564702, 'learning_rate': 0.3426050981153315, 'max_bin': 512, 'm

1480 {'colsample_bytree': 0.3047284117272117, 'learning_rate': 0.37076143172111325, 'max_bin': 128, 'max_depth': 4, 'n_estimators': 1000, 'reg_alpha': 1.6090419851192312, 'reg_lambda': 0.808058239677001} -0.7635419798852953
1490 {'colsample_bytree': 0.6854259740541624, 'learning_rate': 0.3722641293467701, 'max_bin': 128, 'max_depth': 6, 'n_estimators': 500, 'reg_alpha': 1.9181107581462036, 'reg_lambda': 0.0957823976345552} -0.7228363112170433
1500 {'colsample_bytree': 0.3743699065914667, 'learning_rate': 0.37299549438642665, 'max_bin': 64, 'max_depth': 5, 'n_estimators': 500, 'reg_alpha': 0.6061565210618837, 'reg_lambda': 0.6640547389964764} -0.7356067392736287
1510 {'colsample_bytree': 0.6990999863202481, 'learning_rate': 0.09905008435021084, 'max_bin': 256, 'max_depth': 8, 'n_estimators': 500, 'reg_alpha': 2.1020548026168018, 'reg_lambda': 1.149639556195349} -0.6269570827801318
1520 {'colsample_bytree': 0.5380304215476654, 'learning_rate': 0.3450514352454435, 'max_bin': 512, 'max_dep

...

2420 {'colsample_bytree': 0.37543761607821474, 'learning_rate': 0.4719495636001078, 'max_bin': 256, 'max_depth': 4, 'n_estimators': 5000, 'reg_alpha': 1.4706727355945408, 'reg_lambda': 0.62596904567477} -0.8288497922691561
2430 {'colsample_bytree': 1.1409229860457308, 'learning_rate': 0.10883512798387651, 'max_bin': 128, 'max_depth': 5, 'n_estimators': 5000, 'reg_alpha': 2.1396273546742908, 'reg_lambda': 1.8524712541465664} -0.6287480854953802
2440 {'colsample_bytree': 0.24178285637876235, 'learning_rate': 0.1974989466587312, 'max_bin': 256, 'max_depth': 6, 'n_estimators': 5000, 'reg_alpha': 1.5200599375594797, 'reg_lambda': 1.8528850297985127} -0.9099493699741055
2450 {'colsample_bytree': 1.1066260772162178, 'learning_rate': 0.16745104204134292, 'max_bin': 64, 'max_depth': 5, 'n_estimators': 2500, 'reg_alpha': 2.897044681490993, 'reg_lambda': 1.9679703323502813} -0.6461354028278014
2460 {'colsample_bytree': 0.35821965539530104, 'learning_rate': 0.25093772702840994, 'max_bin': 64, 'max

2790 {'colsample_bytree': 1.0505270631133155, 'learning_rate': 0.057312760602202584, 'max_bin': 512, 'max_depth': 7, 'n_estimators': 500, 'reg_alpha': 0.9921191220675227, 'reg_lambda': 0.5587015982742494} -0.5988662952271657
2800 {'colsample_bytree': 0.6398723223150007, 'learning_rate': 0.05874336268608937, 'max_bin': 64, 'max_depth': 7, 'n_estimators': 5000, 'reg_alpha': 0.5473069919843672, 'reg_lambda': 2.7460129044762533} -0.64958222335143
2810 {'colsample_bytree': 0.6650715773038798, 'learning_rate': 0.37958192176838956, 'max_bin': 64, 'max_depth': 7, 'n_estimators': 500, 'reg_alpha': 1.5503323526318287, 'reg_lambda': 2.947465252182804} -0.6927123174758709
2820 {'colsample_bytree': 0.8989923820409143, 'learning_rate': 0.42667850421599496, 'max_bin': 512, 'max_depth': 7, 'n_estimators': 2500, 'reg_alpha': 0.7914363515047668, 'reg_lambda': 2.8897020405276392} -0.6697362872322269
2830 {'colsample_bytree': 0.6444079277922703, 'learning_rate': 0.13351138203842128, 'max_bin': 128, 'max_d

160 {'colsample_bytree': 1.0843022927272863, 'learning_rate': 0.27824947786440285, 'max_bin': 512, 'max_depth': 6, 'n_estimators': 500, 'reg_alpha': 1.003845269184014, 'reg_lambda': 0.7976744744135899} -0.5932635623411017
170 {'colsample_bytree': 0.8019726645129197, 'learning_rate': 0.2269669626787023, 'max_bin': 64, 'max_depth': 8, 'n_estimators': 1000, 'reg_alpha': 0.4027598311385442, 'reg_lambda': 1.803768486560795} -0.6557229733109698
180 {'colsample_bytree': 0.47654056345561585, 'learning_rate': 0.4683024377044817, 'max_bin': 512, 'max_depth': 7, 'n_estimators': 2500, 'reg_alpha': 0.706170617814376, 'reg_lambda': 2.9350820992247577} -0.6811432994350283
190 {'colsample_bytree': 0.2297022424609046, 'learning_rate': 0.08090643383146118, 'max_bin': 256, 'max_depth': 9, 'n_estimators': 2500, 'reg_alpha': 0.10112803283994021, 'reg_lambda': 0.9664791797885969} -0.9092238115995763
200 {'colsample_bytree': 1.158288257207334, 'learning_rate': 0.04207601564699737, 'max_bin': 128, 'max_depth'

530 {'colsample_bytree': 1.0314870759511883, 'learning_rate': 0.15806399087989703, 'max_bin': 64, 'max_depth': 5, 'n_estimators': 5000, 'reg_alpha': 1.8641867467712725, 'reg_lambda': 2.2113358225824844} -0.6601650971486581
540 {'colsample_bytree': 0.5119880409718847, 'learning_rate': 0.4933772137950955, 'max_bin': 256, 'max_depth': 7, 'n_estimators': 1000, 'reg_alpha': 0.7690839516360848, 'reg_lambda': 2.441181695595782} -0.7170860151652249
550 {'colsample_bytree': 0.740346217352623, 'learning_rate': 0.4205451665200144, 'max_bin': 128, 'max_depth': 8, 'n_estimators': 500, 'reg_alpha': 0.8976908716361236, 'reg_lambda': 2.11329061672692} -0.6707718709539784
560 {'colsample_bytree': 0.7644429696253141, 'learning_rate': 0.41509444721166033, 'max_bin': 64, 'max_depth': 7, 'n_estimators': 1000, 'reg_alpha': 2.53358616548739, 'reg_lambda': 1.6167176915659969} -0.6940935037208686
570 {'colsample_bytree': 0.4577988212536887, 'learning_rate': 0.2703149715719283, 'max_bin': 64, 'max_depth': 9, 'n

900 {'colsample_bytree': 1.077923491254735, 'learning_rate': 0.4119616607714157, 'max_bin': 512, 'max_depth': 8, 'n_estimators': 5000, 'reg_alpha': 0.23331042166798854, 'reg_lambda': 0.4534149656289125} -0.6605679004502121
910 {'colsample_bytree': 0.25964508883356935, 'learning_rate': 0.3907053445081901, 'max_bin': 512, 'max_depth': 6, 'n_estimators': 2500, 'reg_alpha': 0.6887718397384638, 'reg_lambda': 0.38827312506866085} -0.9335349973700859
920 {'colsample_bytree': 0.774494001164544, 'learning_rate': 0.15967500324632594, 'max_bin': 256, 'max_depth': 8, 'n_estimators': 5000, 'reg_alpha': 0.40170321191587033, 'reg_lambda': 2.0327638959533414} -0.6154467820297786
930 {'colsample_bytree': 0.2002084998214641, 'learning_rate': 0.4194217247773776, 'max_bin': 64, 'max_depth': 4, 'n_estimators': 500, 'reg_alpha': 0.024274287934912264, 'reg_lambda': 0.8945649257231698} -0.8680173740127708
940 {'colsample_bytree': 0.33592838307974965, 'learning_rate': 0.033180985951953734, 'max_bin': 64, 'max_

In [ ]:
for res_dict in res_dict_list:
    inverse = [(value, key) for key, value in res_dict.items()]
    print(max(inverse))

# bayesian opt

In [11]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

def g_hat(max_bin_exp,
        n_estimators,
        max_depth,
        colsample_bytree,
        learning_rate,
        reg_alpha,
        reg_lambda,):
    estimator_list = np.array([100, 1000, 2000, 5000])
    tmp = abs(estimator_list - n_estimators)
    n_estimators = estimator_list[np.argmin(tmp)]
    return g(
        2**int(max_bin_exp),
        n_estimators,
        int(max_depth),
        colsample_bytree,
        learning_rate,
        reg_alpha,
        reg_lambda,
    )

pbounds = {
    'max_bin_exp': (4, 10),
    'n_estimators': (100, 5000),
    "max_depth": (1, 6),
    "colsample_bytree": (0.2, 1),
    'learning_rate': (0.001, 0.5),
    'reg_alpha': (0, 3),
    'reg_lambda': (0, 3),
}

b_res_list = []
for n in range(n_experiment):
    optimizer = BayesianOptimization(
        f=g_hat,
        pbounds=pbounds,
        random_state=n*128,
        allow_duplicate_points=False
    )
    logger = JSONLogger(path=f"/home/xyz/data/haoyan/bandits/logs/bayesian_results_{n*128}.log")
    optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

    optimizer.maximize(init_points=3, n_iter=597)
    b_res_list.append(optimizer.max)

In [ ]:
[(x['target'], str({'u': int(int(x['params']['u'])/n_interval_u) * n_interval_u + 1, 'v': int(int(x['params']['v'])/n_interval_v) * n_interval_v + 1, 'w': x['params']['w'], 'x': int(int(x['params']['x'])/n_interval_x) * n_interval_x + 1, 'y': x['params']['y'], 'z': x['params']['z']})) for x in b_res_list]

[(0.2768580338893678,
  "{'u': 4, 'v': 7, 'w': 5.116789806249014, 'x': 13, 'y': 10.080905428791336, 'z': 0.9911011711627078}"),
 (0.1402183484847389,
  "{'u': 4, 'v': 7, 'w': 5.011204786829208, 'x': 1, 'y': 10.000305831311975, 'z': 0.9868788787576984}"),
 (0.27722306853469403,
  "{'u': 4, 'v': 7, 'w': 4.902417985793109, 'x': 13, 'y': 9.945820186243077, 'z': 0.924097238551662}"),
 (0.2768119314094586,
  "{'u': 4, 'v': 7, 'w': 4.885291040699603, 'x': 13, 'y': 10.081867600308184, 'z': 1.0257453084676125}"),
 (0.2768976627971634,
  "{'u': 4, 'v': 7, 'w': 5.09077481582764, 'x': 13, 'y': 9.937800034531513, 'z': 0.9109290708881388}"),
 (0.2799674220581756,
  "{'u': 4, 'v': 7, 'w': 4.948148565613696, 'x': 13, 'y': 10.012700410959875, 'z': 0.9925553408392699}"),
 (0.2753907393071401,
  "{'u': 4, 'v': 7, 'w': 5.075218248621907, 'x': 13, 'y': 9.860434685868848, 'z': 0.9419097111446684}"),
 (0.13862511632972735,
  "{'u': 4, 'v': 7, 'w': 5.048247231101259, 'x': 1, 'y': 10.066125798110978, 'z': 0.89

# rl

In [ ]:
n_trials = 3000

params_list_dict = {
    "max_bin": [2**6, 2**7, 2**8, 2**9],
    "max_depth": [4, 5, 6, 7, 8, 9],
    "n_estimators": [500, 1000, 2500, 5000],
    "colsample_bytree": np.linspace(0.2, 1, 10),
    'learning_rate': np.linspace(0.001, 0.5, 30),
    'reg_alpha': np.linspace(0, 3, 30),
    'reg_lambda': np.linspace(0, 3, 30),
}

scores_list_ori = []
actions_list_ori = []

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/rl_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/rl_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/rl_results.txt', 'a') as f:

    for n in range(n_experiment):
        bandit = bd.GaussianBanditOri(
            g,
            params_list_dict=params_list_dict,
        )

        policy = bd.SoftmaxPolicy()
        agent = bd.GradientAgent(bandit, policy, alpha=0.1, baseline=True, increase_rate=0)
        env = bd.Environment(bandit, agent, 'Gradient Agents', random_state=n*128)
        scores, actions = env.run(n_trials, file=f)

        idx = np.argmax(scores)

        scores_list_ori.append(scores)
        actions_list_ori.append(actions)

0
0.1
((1024, 2000, 6, 0.9111111111111112, 0.3279310344827586, 2.2758620689655173, 2.0689655172413794), None) -0.6260685358179585
1
0.1
((1024, 100, 5, 0.37777777777777777, 0.3107241379310345, 0.20689655172413793, 1.5517241379310345), None) -0.8328349966322386
2
0.1
((64, 5000, 6, 0.2888888888888889, 0.15586206896551724, 0.41379310344827586, 2.0689655172413794), None) -0.8418240917834782
3
0.1
((128, 2000, 3, 0.2, 0.20748275862068966, 2.793103448275862, 2.2758620689655173), None) -0.8793157143802965
4
0.1
((256, 2000, 5, 0.37777777777777777, 0.19027586206896552, 2.4827586206896552, 1.9655172413793103), None) -0.7167884216563309
5
0.1
((256, 1000, 4, 0.7333333333333334, 0.2246896551724138, 2.793103448275862, 3.0), None) -0.6198023924045138
6
0.1
((32, 1000, 3, 0.8222222222222222, 0.4655862068965517, 0.5172413793103449, 0.5172413793103449), None) -0.7707729103842986
7
0.1
((128, 100, 2, 0.7333333333333334, 0.001, 1.7586206896551724, 1.4482758620689655), None) -4.656989500353108
8
0.1
((6

0.1
((16, 2000, 4, 0.2, 0.3623448275862069, 2.689655172413793, 0.3103448275862069), None) -3.1434687779822856
67
0.1
((16, 5000, 4, 0.7333333333333334, 0.17306896551724138, 0.8275862068965517, 2.1724137931034484), None) -1.0883461385813915
68
0.1
((128, 5000, 6, 0.2888888888888889, 0.4483793103448276, 1.8620689655172413, 2.689655172413793), None) -0.8829803433260414
69
0.1
((1024, 1000, 6, 0.5555555555555556, 0.24189655172413793, 2.4827586206896552, 1.4482758620689655), None) -0.6459754424243762
70
0.1
((256, 2000, 4, 0.9111111111111112, 0.06982758620689655, 2.1724137931034484, 0.10344827586206896), None) -0.6265455392383181
71
0.1
((16, 2000, 4, 0.2888888888888889, 0.4139655172413793, 0.5172413793103449, 0.0), None) -1.3235876037238112
72
0.1
((512, 1000, 5, 0.9111111111111112, 0.3967586206896552, 2.1724137931034484, 0.20689655172413793), None) -0.6601016519479754
73
0.1
((1024, 5000, 5, 0.8222222222222222, 0.08703448275862069, 0.20689655172413793, 2.586206896551724), None) -0.5977373

0.1
((1024, 1000, 5, 0.7333333333333334, 0.17306896551724138, 1.1379310344827587, 0.41379310344827586), None) -0.6130731549817561
134
0.1
((32, 100, 2, 0.2888888888888889, 0.2763103448275862, 0.3103448275862069, 2.2758620689655173), None) -2.2348664159545666
135
0.1
((64, 2000, 5, 0.4666666666666667, 0.19027586206896552, 3.0, 0.5172413793103449), None) -0.7251769841447446
136
0.1
((16, 1000, 6, 0.9111111111111112, 0.17306896551724138, 1.2413793103448276, 2.3793103448275863), None) -1.0351576604946444
137
0.1
((64, 100, 4, 0.7333333333333334, 0.3967586206896552, 0.6206896551724138, 2.793103448275862), None) -0.6688710296418904
138
0.1
((128, 100, 3, 1.0, 0.19027586206896552, 0.7241379310344828, 0.6206896551724138), None) -0.8355957175729757
139
0.1
((64, 100, 4, 0.2, 0.24189655172413793, 2.896551724137931, 1.4482758620689655), None) -1.6234752791696092
140
0.1
((256, 100, 6, 0.2888888888888889, 0.1386551724137931, 1.8620689655172413, 0.9310344827586207), None) -0.9838842335657955
141
0.

0.1
((64, 2000, 4, 0.2, 0.01820689655172414, 2.4827586206896552, 1.0344827586206897), None) -1.5049309910516715
200
0.1
((512, 2000, 4, 0.37777777777777777, 0.10424137931034483, 2.0689655172413794, 1.9655172413793103), None) -0.6658038511202422
201
0.1
((256, 1000, 2, 0.5555555555555556, 0.19027586206896552, 0.7241379310344828, 1.4482758620689655), None) -0.7442135368570504
202
0.1
((16, 5000, 6, 0.2888888888888889, 0.3107241379310345, 1.7586206896551724, 2.689655172413793), None) -1.4252912974035574
203
0.1
((256, 100, 2, 0.2, 0.3279310344827586, 2.896551724137931, 2.689655172413793), None) -2.5765958818032013
204
0.1
((16, 5000, 4, 0.8222222222222222, 0.43117241379310345, 0.41379310344827586, 2.0689655172413794), None) -1.1573290225671455
205
0.1
((16, 1000, 3, 0.9111111111111112, 0.12144827586206897, 1.1379310344827587, 1.5517241379310345), None) -1.2582422880303965
206
0.1
((128, 2000, 2, 0.6444444444444445, 0.3623448275862069, 2.4827586206896552, 0.8275862068965517), None) -0.7015

0.1
((32, 2000, 4, 1.0, 0.17306896551724138, 0.3103448275862069, 1.2413793103448276), None) -0.7497864386277366
266
0.1
((1024, 5000, 6, 0.2, 0.24189655172413793, 2.2758620689655173, 2.3793103448275863), None) -0.9694537991393009
267
0.1
((64, 2000, 2, 0.9111111111111112, 0.43117241379310345, 1.8620689655172413, 1.8620689655172413), None) -0.7056774155985168
268
0.1
((1024, 1000, 5, 0.2, 0.10424137931034483, 0.3103448275862069, 1.7586206896551724), None) -0.8573869944812265
269
0.1
((512, 1000, 3, 0.5555555555555556, 0.37955172413793103, 0.7241379310344828, 1.3448275862068966), None) -0.6789993165563207
270
0.1
((64, 2000, 3, 0.8222222222222222, 0.29351724137931035, 2.793103448275862, 0.6206896551724138), None) -0.6839185483779131
271
0.1
((16, 2000, 4, 0.2888888888888889, 0.4139655172413793, 2.4827586206896552, 1.4482758620689655), None) -1.3204220489222411
272
0.1
((64, 5000, 3, 0.8222222222222222, 0.4655862068965517, 2.896551724137931, 1.3448275862068966), None) -0.7640960120542019


...

0.1
((512, 1000, 3, 0.5555555555555556, 0.1386551724137931, 0.6206896551724138, 3.0), None) -0.6738574899603488
251
0.1
((32, 100, 5, 0.2888888888888889, 0.001, 0.41379310344827586, 2.793103448275862), None) -4.6605426274835215
252
0.1
((512, 100, 4, 0.2888888888888889, 0.3623448275862069, 0.6206896551724138, 1.0344827586206897), None) -0.8767266060469633
253
0.1
((32, 5000, 3, 1.0, 0.08703448275862069, 2.793103448275862, 0.20689655172413793), None) -0.7429487315674288
254
0.1
((16, 1000, 5, 0.8222222222222222, 0.19027586206896552, 1.2413793103448276, 2.689655172413793), None) -1.0517050458737935
255
0.1
((256, 100, 6, 0.2888888888888889, 0.12144827586206897, 1.6551724137931034, 2.896551724137931), None) -1.0276292603048494
256
0.1
((64, 100, 4, 1.0, 0.3623448275862069, 2.586206896551724, 1.9655172413793103), None) -0.6317370445386065
257
0.1
((1024, 100, 3, 0.8222222222222222, 0.25910344827586207, 2.0689655172413794, 2.1724137931034484), None) -0.8073495081479518
258
0.1
((32, 1000, 4

0.1
((256, 100, 3, 0.8222222222222222, 0.25910344827586207, 1.5517241379310345, 0.8275862068965517), None) -0.7963603194503296
318
0.1
((1024, 1000, 4, 0.37777777777777777, 0.24189655172413793, 1.2413793103448276, 2.689655172413793), None) -0.6846141769034692
319
0.1
((32, 100, 2, 0.37777777777777777, 0.5, 3.0, 2.2758620689655173), None) -1.6838373549111347
320
0.1
((128, 5000, 5, 0.5555555555555556, 0.20748275862068966, 0.8275862068965517, 1.8620689655172413), None) -0.6986671263933028
321
0.1
((256, 5000, 3, 1.0, 0.4483793103448276, 1.0344827586206897, 2.4827586206896552), None) -0.7200505861816177
322
0.1
((512, 1000, 5, 0.37777777777777777, 0.3107241379310345, 2.0689655172413794, 3.0), None) -0.699354004497475
323
0.1
((64, 2000, 2, 1.0, 0.4483793103448276, 2.2758620689655173, 0.5172413793103449), None) -0.7118250286885005
324
0.1
((32, 1000, 6, 0.37777777777777777, 0.2246896551724138, 1.7586206896551724, 1.9655172413793103), None) -0.8521338136556911
325
0.1
((128, 100, 3, 0.55555

0.1
((256, 5000, 2, 0.37777777777777777, 0.10424137931034483, 2.586206896551724, 2.1724137931034484), None) -0.7412676933380414
384
0.1
((512, 1000, 3, 1.0, 0.5, 0.8275862068965517, 2.2758620689655173), None) -0.6658537372116751
385
0.1
((256, 5000, 4, 0.5555555555555556, 0.29351724137931035, 0.0, 2.3793103448275863), None) -0.7114154139481079
386
0.1
((64, 100, 6, 0.37777777777777777, 0.24189655172413793, 0.0, 2.3793103448275863), None) -0.7835094865193915
387
0.1
((128, 100, 4, 0.6444444444444445, 0.37955172413793103, 0.3103448275862069, 0.7241379310344828), None) -0.6903573071195269
388
0.1
((256, 100, 2, 0.8222222222222222, 0.3279310344827586, 0.0, 2.0689655172413794), None) -1.0233757000706214
389
0.1
((64, 1000, 6, 0.9111111111111112, 0.19027586206896552, 2.793103448275862, 1.6551724137931034), None) -0.6376681099774895
390
0.1
((128, 100, 3, 1.0, 0.3107241379310345, 1.2413793103448276, 1.2413793103448276), None) -0.743894241682483
391
0.1
((256, 2000, 5, 0.37777777777777777, 0.5

0.1
((16, 2000, 4, 0.9111111111111112, 0.20748275862068966, 1.8620689655172413, 0.7241379310344828), None) -1.0870106469624035
451
0.1
((1024, 2000, 5, 0.8222222222222222, 0.15586206896551724, 2.4827586206896552, 1.0344827586206897), None) -0.6011991885637359
452
0.1
((512, 2000, 3, 0.6444444444444445, 0.052620689655172415, 2.3793103448275863, 2.1724137931034484), None) -0.6509203497108933
453
0.1
((16, 1000, 3, 0.5555555555555556, 0.2246896551724138, 1.6551724137931034, 0.5172413793103449), None) -1.2736633029013653
454
0.1
((16, 2000, 2, 0.8222222222222222, 0.08703448275862069, 0.9310344827586207, 0.20689655172413793), None) -1.7516956391550065
455
0.1
((32, 2000, 4, 0.37777777777777777, 0.48279310344827586, 1.4482758620689655, 2.0689655172413794), None) -0.8752010279756649
456
0.1
((512, 1000, 6, 0.5555555555555556, 0.29351724137931035, 0.3103448275862069, 2.586206896551724), None) -0.6575992973995114
457
0.1
((64, 1000, 3, 0.5555555555555556, 0.001, 1.7586206896551724, 0.0), None) 

0.1
((32, 2000, 2, 0.6444444444444445, 0.29351724137931035, 0.0, 2.4827586206896552), None) -0.8103826461143478
517
0.1
((128, 5000, 3, 0.6444444444444445, 0.43117241379310345, 1.2413793103448276, 0.10344827586206896), None) -0.7691932451575786
518
0.1
((128, 1000, 2, 0.6444444444444445, 0.24189655172413793, 0.9310344827586207, 3.0), None) -0.7239669985331333
519
0.1
((512, 5000, 2, 0.5555555555555556, 0.08703448275862069, 1.9655172413793103, 0.0), None) -0.7091701003759123
520
0.1
((1024, 2000, 6, 0.6444444444444445, 0.03541379310344828, 0.10344827586206896, 1.1379310344827587), None) -0.6048283081523658
521
0.1
((512, 1000, 2, 0.4666666666666667, 0.29351724137931035, 2.896551724137931, 1.1379310344827587), None) -0.720744961880223
522
0.1
((16, 2000, 6, 0.2888888888888889, 0.4139655172413793, 0.0, 0.9310344827586207), None) -1.3879239488935968
523
0.1
((16, 2000, 3, 0.37777777777777777, 0.19027586206896552, 1.8620689655172413, 0.6206896551724138), None) -1.416350976850779
524
0.1
((5

In [ ]:
for scores, actions in zip(scores_list_ori, actions_list_ori):
    print(actions[np.argmax(scores)][0], scores[np.argmax(scores)])

(335, 4.348633064005995) 0.03436411662463359
(375, 8.056192227792936) 0.03387046171330028
(267, 12.158172579216787) 0.03839516208255515
(412, 7.819539515210791) 0.03255006780027559
(449, 11.44821444147035) 0.033082805223584796
(365, 5.610780864444103) 0.032638915566141534
(426, 4.664170014115522) 0.0322694725026607
(480, 3.9542118763690866) 0.03496507206821923
(391, 8.68726612801199) 0.03557410140924732
(124, 9.239455790703662) 0.04142491925257696


our hybrid optimization method takes advantage of reinforcement learning to handle the discrete parameters, which successfully reduces the dimension of the parameter space for Bayesian Optimization. The technique improves the accuracy
of the solutions, compared with Bayesian Optimization and Random Search. The overhead of the one step Bayesian Optimization
is overcomed by highly reducing the iteration steps of Bayesian Optimization in the hybrid method because of the 
dimension reduction. The overall iteration steps is also compressed compared with Random Search to reach the global
minimizer.